In [1]:
import pandas as pd

In [2]:
import rbot
from rbot import BaseAgent
from rbot import BinanceMarket
from rbot import BackTester
from rbot import time_string
from rbot import Market
from rbot import DAYS_BEFORE
from rbot.chart import Chart

In [3]:
class Agent(BaseAgent):
    def __init__(self, param_K=1.6):
            self.K = param_K                           # パラメターKを設定する。

    def clock_interval(self):
        #//return 60*60*2
        interval = 60*60*2
        return interval
    
    #def on_tick(self, time_us, session, price, side, size):
    #    print(time_us, price, side, size)

    def on_clock(self, time_us, session):
        ohlcv_df = session.ohlcv(60*60*2, 6)       # ６本の足を取得。 最新は６番目。

        if len(ohlcv_df.index) < 6:                 # データが過去６本分そろっていない場合はなにもせずリターン
            return 

        ohlcv_df["range"] = ohlcv_df["high"] - ohlcv_df["low"]      # レンジを計算

        ohlcv_latest = ohlcv_df[-2:-1]     # 最新足１本
        ohlcv_last_5 = ohlcv_df[:-2]       # 過去５本足

        range_width = ohlcv_last_5["range"].mean()      #　過去５本足のレンジの平均値

        # Long/Short判定
        detect_short = range_width * self.K < ohlcv_latest["high"][0] - ohlcv_latest["open"][0]
        detect_long  = range_width * self.K < ohlcv_latest["open"][0] - ohlcv_latest["low"][0]

        #　執行方法（順報告のポジションがあったら保留。逆方向のポジションがのこっていたらドテン）
        if detect_long:
            #    print("position", session.long_pos_size, session.short_pos_size)            
            #        print("make long")
            if not session.long_position_size:
            #    print("makeorder")                                    
                if not session.short_position_size:

                    session.make_order("Buy", session.buy_board_edge_price, 0.01, 600, "Open Long")    
                else:

                    session.make_order("Buy", session.buy_board_edge_price, 0.02, 600, "doten Long")    
            else:
                pass

        if detect_short:
            #print("make short")            
            if not session.short_position_size:
                #print("makeorder")                                                      
                if not session.long_position_size:
                    session.make_order("Sell", session.sell_board_edge_price, 0.01, 600, "Open Short") 
                else:
                    session.make_order("Sell", session.sell_board_edge_price, 0.02, 600, "Doten Short") 
            else:
                pass

    def on_update(self, time, session, result):
        #print(result)
        pass


In [4]:
back_tester = BackTester("BN", "BTCBUSD", False)
back_tester.maker_fee_rate = 0.1 * 0.01  # 0.1%
back_tester.maker_fee_rate

0.001

In [5]:


print("start")
r = back_tester.run(Agent(), rbot.DAYS_BEFORE(10), 0)
#r = back_tester.run(Agent(), 0, 0)
print("end")




start
end


In [6]:
import rbot
df = rbot.result_to_df(r)

In [7]:
df

,update_time,order_id,sub_id,order_side,post_only,status,open_price,open_size,open_volume,close_price,...,order_price,order_size,order_volume,profit,fee,total_profit,position_change,message,sum_profit,position
create_time,,,,,,,,,,,,,,,,,,,,,
2022-12-01 13:59:59.781000+00:00,2022-12-01 14:00:00.159000+00:00,0000-0001,0,Buy,True,Open,17090.70,0.01,170.9070,0.00,...,17090.70,0.01,170.9070,0.0000,0.170907,-0.170907,0.01,Open Long,-0.170907,0.01
2022-12-01 13:59:59.781000+00:00,2022-12-01 14:09:21.167000+00:00,0000-0002,0,Sell,True,Close,17090.70,0.01,170.9070,17091.37,...,17091.37,0.01,170.9137,0.0067,0.170914,-0.164214,-0.01,Open Short,-0.335121,0.00
2022-12-01 15:59:59.892000+00:00,2022-12-01 16:00:00.075000+00:00,0000-0003,0,Buy,True,Open,16978.62,0.01,169.7862,0.00,...,16978.62,0.01,169.7862,0.0000,0.169786,-0.169786,0.01,Open Long,-0.504907,0.01
2022-12-02 13:59:59.784000+00:00,2022-12-02 14:00:02.414000+00:00,0000-0004,0,Sell,True,Close,16978.62,0.01,169.7862,16892.18,...,16892.18,0.01,168.9218,-0.8644,0.168922,-1.033322,-0.01,Doten Short,-1.538229,0.00
2022-12-02 13:59:59.784000+00:00,2022-12-02 14:00:02.414000+00:00,0000-0004,1,Sell,True,Open,16892.18,0.01,168.9218,0.00,...,16892.18,0.01,168.9218,0.0000,0.168922,-0.168922,-0.01,Doten Short,-1.707150,-0.01
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0005,0,Buy,True,Close,16892.18,0.01,168.9218,16949.21,...,16949.21,0.01,169.4921,-0.5703,0.169492,-0.739792,0.01,doten Long,-2.446943,0.00
2022-12-03 21:59:59.022000+00:00,2022-12-03 22:00:02.010000+00:00,0000-0005,1,Buy,True,Open,16949.21,0.01,169.4921,0.00,...,16949.21,0.01,169.4921,0.0000,0.169492,-0.169492,0.01,doten Long,-2.616435,0.01
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0006,0,Sell,True,Close,16949.21,0.01,169.4921,16968.66,...,16968.66,0.01,169.6866,0.1945,0.169687,0.024813,-0.01,Doten Short,-2.591621,0.00
2022-12-04 01:59:59.783000+00:00,2022-12-04 02:00:05.814000+00:00,0000-0006,1,Sell,True,Open,16968.66,0.01,169.6866,0.00,...,16968.66,0.01,169.6866,0.0000,0.169687,-0.169687,-0.01,Doten Short,-2.761308,-0.01


In [8]:
market = Market.open("BN", "BTCBUSD")
ohlcv = market.ohlcv(rbot.NOW()-rbot.DAYS(12), 0, 60*60)


In [9]:
chart = Chart(800, 400, ohlcv)
chart.show()

Loading BokehJS ...

main


In [10]:
df = result_to_df(r)
df

NameError: name 'result_to_df' is not defined

In [ ]:
bn = BinanceMarket("BTCBUSD")
#bn.download(15)


In [ ]:
bn

In [ ]:
rbot.init_debug_log()
bn.download(10)

In [ ]:
bn.info()

In [ ]:
rbot.init_debug_log()
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 600))
#print(bn.ohlcvv(1669593599839000 - 60*60*50*1000_000, 1669593599839000, 601))
bn.ohlcvv(0, 1669593599839000, 601)

In [ ]:
bn.ohlcv(0, 0, 600)

In [ ]:
result_to_df(r)

In [ ]:
r[0].update_time